In [1]:
%config SqlMagic.displaycon = False
%load_ext sql
%sql url

In [2]:
%%sql
WITH min_max_drug_id AS (
	SELECT
		 de.drug_concept_id
		,min(drug_exposure_start_date) start_date
		,max(drug_exposure_end_date) end_date
	FROM de.drug_exposure de
	WHERE person_id = 1891866
	GROUP BY de.drug_concept_id 
)

SELECT 
	 mmdi.drug_concept_id
	,mmdi.start_date
	,mmdi.end_date
	,ed.total_exposure_days
FROM min_max_drug_id mmdi
LEFT JOIN (
    -- exposure days per drug_concept_id
	SELECT
		 drug_concept_id 
		,SUM(drug_exposure_end_date - drug_exposure_start_date  + 1) AS total_exposure_days
	FROM de.drug_exposure
	WHERE (person_id = 1891866)
	AND drug_exposure_id NOT IN (
	-- 같은 약이며, 복용 시작일이 같지만, 복용 종료일이 다른 경우의 drug_exposure_id 제외 (중복되므로)
		SELECT
			t1.drug_exposure_id 
		FROM (
			SELECT 
				 drug_exposure_id 
				,drug_concept_id 
				,drug_exposure_start_date 
				,drug_exposure_end_date 
				,ROW_NUMBER() OVER (
					PARTITION BY drug_concept_id, drug_exposure_start_date 
					ORDER BY drug_exposure_end_date DESC) AS row_num
			FROM de.drug_exposure de 
			WHERE person_id = 1891866
		) AS t1
		WHERE t1.row_num > 1
	)
	GROUP BY drug_concept_id 
) ed
	ON mmdi.drug_concept_id = ed.drug_concept_id
ORDER BY total_exposure_days

5 rows affected.


drug_concept_id,start_date,end_date,total_exposure_days
40213227,1993-01-05,1993-01-05,1
40213154,1989-09-12,1998-07-07,10
19030765,1988-10-18,1998-10-05,1214
1539463,1990-03-13,1998-03-11,5484
19009384,1959-12-01,1998-10-06,14421
